## A toy example of using the dgrid_components.py module in risktables, for both plots inside a jupyter notebook, and to run a web app from a jupyter notebook

In [48]:
from IPython.display import display
import dash
import sys,os
if  not os.path.abspath('./') in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not os.path.abspath('../') in sys.path:
    sys.path.append(os.path.abspath('../'))

import datetime
import pandas as pd
import numpy as np
import importlib
import traceback

from dashgrid import dgrid_components as dgc
from dashgrid import dash_df_components as ddfc
import dash_html_components as html

import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
import pathlib
SYSTEM_HOME = pathlib.Path.home()

### Define a logger that all components can use

In [2]:
logger = dgc.init_root_logger('logfile.log','INFO')

### Define several Dash Table examples

In [3]:
# This Markdown component allows you to use markdown in html divs
# create row 1

def make_title(title_markdown,style=None):
    tm_text = title_markdown
    if tm_text is None:
        tm_text = '''
                    # Dash DataFrame App
                    This app displays Pandas DataFrames, Plotly Graphs and other html components 
                    like file inputs, dropdown lists, and text input fields, and general divs.

                    '''
    input_style = 'white' if style is None else style
    top_markdown = dgc.dcc.Markdown(tm_text
                        ,style={'color':input_style})
    # you need to define some css styles as well
    STYLE_TITLE={
        'line-height': '20px',
        'textAlign': 'center',
        'background-color':'#47bacc',
        'color':'#FFFFF9',
        'vertical-align':'middle',
    } 

    top_div = html.Div([top_markdown],style=STYLE_TITLE,id='top_div')
    return top_div


In [ ]:
import importlib
importlib.reload(dgc)
importlib.reload(ddfc)


### DataFrameApp

In [8]:

title_comp = make_title("My example app")
input_div = dgc.DivComponent('input_div',initial_children="Enter a number")
input_comp = dgc.InputBox('input_comp',init_value=1,input_type='number')

x = np.arange(1,10,.1).round(4)
y =  x + np.random.normal(size=len(x))*.1
y = y.round(4)
df = pd.DataFrame({'x':x,'y':y})
scale_y = 1
y2 = x * scale_y + np.random.normal(size=len(x))*.1
df2 = pd.DataFrame({'x':x,'y':y2})
init_data = {"scale_y":scale_y,"df":df.to_dict('rows'),'df2':df2.to_dict('rows')}



table_1_comp = ddfc.TableInput('table_1',initial_data=df,title="Test Table 1 From Store")

def store_transform(input_list):
    if len(input_list)<1:
        return {}
    if input_list[0] is None:
        return {}
    dict_df = input_list[0]
    scale_y = 1 if input_list[1] is None else float(str(input_list[1]))
    df1 = dgc.make_df(dict_df)
    df2 = df1.copy()
    df2.y = df2.y * scale_y
    dict_df2 = df2.to_dict('rows')
    r = {'scale_y':scale_y,"df":dict_df,"df2":dict_df2}
    return r
    
    
storage_input_tuples = [table_1_comp.output_data_tuple,input_comp.output_data_tuple]    
storage_comp = dgc.StoreComponent('store_comp',storage_input_tuples,
        create_data_dictionary_from_df_transformer=store_transform,
        initial_data=init_data)

table_2_comp = ddfc.TableFromStore('table_2',storage_comp,'df2',title="Test Table 2 From Store")

xy_1_comp = dgc.XyGraphComponent('xy_1_comp',table_1_comp,x_column='x',plot_bars=False,title="XY Graph 1")
# xy_2_comp = dgc.XyGraphComponent('xy_2_comp',table_2_comp,x_column='x',plot_bars=False)
xy_2_comp = ddfc.XyGraphFromStore('xy_2_comp',storage_comp,'df2','x',title="XY Graph 2")

app_component_list = [title_comp,
                      input_div,input_comp,
                      storage_comp,
                      xy_1_comp,xy_2_comp,
                      table_1_comp,table_2_comp]
gtcl = ['1fr','1fr 1fr','1fr','1fr 1fr','1fr 1fr']
app_to_use = dgc.dash.Dash()
app = dgc.make_app(app_component_list,grid_template_columns_list=gtcl,app=app_to_use)    
app.run_server(host='127.0.0.1',port=8500)

2020-01-24 10:29:07,461 - werkzeug - INFO - 127.0.0.1 - - [24/Jan/2020 10:29:07] "POST /_dash-update-component HTTP/1.1" 200 -
2020-01-24 10:29:07,515 - werkzeug - INFO - 127.0.0.1 - - [24/Jan/2020 10:29:07] "POST /_dash-update-component HTTP/1.1" 200 -
2020-01-24 10:29:07,551 - werkzeug - INFO - 127.0.0.1 - - [24/Jan/2020 10:29:07] "POST /_dash-update-component HTTP/1.1" 200 -


store_transform input_list: [[{'x': 1, 'y': 1.0517}, {'x': 1.1, 'y': 1.1453}, {'x': 1.2, 'y': 1.2198}, {'x': 1.3, 'y': 1.368}, {'x': 1.4, 'y': 1.5455}, {'x': 1.5, 'y': 1.5013}, {'x': 1.6, 'y': 1.4671}, {'x': 1.7, 'y': 1.6433}, {'x': 1.8, 'y': 1.7985}, {'x': 1.9, 'y': 2.128}, {'x': 2, 'y': 1.9937}, {'x': 2.1, 'y': 2.2276}, {'x': 2.2, 'y': 1.9371}, {'x': 2.3, 'y': 2.2348}, {'x': 2.4, 'y': 2.4101}, {'x': 2.5, 'y': 2.5516}, {'x': 2.6, 'y': 2.5904}, {'x': 2.7, 'y': 2.762}, {'x': 2.8, 'y': 2.794}, {'x': 2.9, 'y': 2.9512}, {'x': 3, 'y': 3.0009}, {'x': 3.1, 'y': 3.0381}, {'x': 3.2, 'y': 3.3089}, {'x': 3.3, 'y': 3.3241}, {'x': 3.4, 'y': 3.618}, {'x': 3.5, 'y': 3.5823}, {'x': 3.6, 'y': 3.5633}, {'x': 3.7, 'y': 3.6051}, {'x': 3.8, 'y': 3.7999}, {'x': 3.9, 'y': 3.7904}, {'x': 4, 'y': 3.8134}, {'x': 4.1, 'y': 4.0882}, {'x': 4.2, 'y': 4.2109}, {'x': 4.3, 'y': 4.5414}, {'x': 4.4, 'y': 4.475}, {'x': 4.5, 'y': 4.6331}, {'x': 4.6, 'y': 4.588}, {'x': 4.7, 'y': 4.7906}, {'x': 4.8, 'y': 4.7117}, {'x': 4.9,

2020-01-24 10:29:09,785 - werkzeug - INFO - 127.0.0.1 - - [24/Jan/2020 10:29:09] "POST /_dash-update-component HTTP/1.1" 200 -
2020-01-24 10:29:09,840 - werkzeug - INFO - 127.0.0.1 - - [24/Jan/2020 10:29:09] "POST /_dash-update-component HTTP/1.1" 200 -
2020-01-24 10:29:09,918 - werkzeug - INFO - 127.0.0.1 - - [24/Jan/2020 10:29:09] "POST /_dash-update-component HTTP/1.1" 200 -


store_transform input_list: [[{'x': 1, 'y': 1.0517}, {'x': 1.1, 'y': 1.1453}, {'x': 1.2, 'y': 1.2198}, {'x': 1.3, 'y': 1.368}, {'x': 1.4, 'y': 1.5455}, {'x': 1.5, 'y': 1.5013}, {'x': 1.6, 'y': 1.4671}, {'x': 1.7, 'y': 1.6433}, {'x': 1.8, 'y': 1.7985}, {'x': 1.9, 'y': 2.128}, {'x': 2, 'y': 1.9937}, {'x': 2.1, 'y': 2.2276}, {'x': 2.2, 'y': 1.9371}, {'x': 2.3, 'y': 2.2348}, {'x': 2.4, 'y': 2.4101}, {'x': 2.5, 'y': 2.5516}, {'x': 2.6, 'y': 2.5904}, {'x': 2.7, 'y': 2.762}, {'x': 2.8, 'y': 2.794}, {'x': 2.9, 'y': 2.9512}, {'x': 3, 'y': 3.0009}, {'x': 3.1, 'y': 3.0381}, {'x': 3.2, 'y': 3.3089}, {'x': 3.3, 'y': 3.3241}, {'x': 3.4, 'y': 3.618}, {'x': 3.5, 'y': 3.5823}, {'x': 3.6, 'y': 3.5633}, {'x': 3.7, 'y': 3.6051}, {'x': 3.8, 'y': 3.7999}, {'x': 3.9, 'y': 3.7904}, {'x': 4, 'y': 3.8134}, {'x': 4.1, 'y': 4.0882}, {'x': 4.2, 'y': 4.2109}, {'x': 4.3, 'y': 4.5414}, {'x': 4.4, 'y': 4.475}, {'x': 4.5, 'y': 4.6331}, {'x': 4.6, 'y': 4.588}, {'x': 4.7, 'y': 4.7906}, {'x': 4.8, 'y': 4.7117}, {'x': 4.9,

In [ ]:
def c1(a,b,**kwargs):
    def c2(c):
        print(a,b,c,c3(**kwargs))
    return c2

def c3(**kwargs):
    if 'd' in kwargs.keys():
        return f"from c3.d {kwargs['d']}"

a1 = 1
a2 = 2
f1 = c1(a1,a2,d='d1')
f1(3)
a1 = 4
a2 = 5
f1(5)

In [ ]:
import importlib
importlib.reload(dgc)
importlib.reload(ddfc)


x = np.arange(1,10,.1).round(4)
y =  x + np.random.normal(size=len(x))*.1
y = y.round(4)
df = pd.DataFrame({'x':x,'y':y})
scale_y = -
y2 = x * scale_y + np.random.normal(size=len(x))*.1
df2 = pd.DataFrame({'x':x,'y':y2})
my_input_dict = {'df':df.to_dict('rows'),'df2':df2.to_dict('rows')}


In [ ]:
table_example = ddfc.TableInput('r1c1',initial_data=df,title="Table r1c1")
xy_example = ddfc.XYGraphSimple('r1c2',initial_data=df,x_column='x',title="Graph r1c2")  


In [ ]:
te_clone = table_example.clone('r2c1',initial_data=df2,title=None)
xy_clone = xy_example.clone('r2c2',initial_data=df2,title=None)


In [ ]:
comp_list = [table_example,xy_example,te_clone,xy_clone]
app = dgc.make_app(comp_list,grid_template_columns_list=['1fr 1fr','1fr 1fr'])    
app.run_server(host='127.0.0.1',port=8500)    


## Building new reactive system
1. must create a full class
2. the input components have to be simplified
3. what happens if i use file upload as one of my inputs.
4. it might help to have a bunch of example apps, with nothing by input components

In [114]:
import importlib
importlib.reload(dgc)
importlib.reload(ddfc)

# create data
x = np.arange(1,10,.1).round(4)
y =  x + np.random.normal(size=len(x))*.1
y = y.round(4)
df = pd.DataFrame({'x':x,'y':y})

title_table = 'Initial Table'
title_graph = 'Initial Graph'
table_example = ddfc.TableInput('init_table',initial_data=df,title=title_table)
xy_example = ddfc.XYGraphSimple('init_graph',initial_data=df,x_column='x',title=title_graph)  

my_row_comps = [table_example,xy_example]
my_row_layout = '1fr 1fr'
my_input_dict = {'df':df}

title_comp = make_title("My example app")
input_div = dgc.DivComponent('input_div',initial_children="Enter a number")
input_comp = ddfc.TextBoxInput('input_box',init_value=1,input_type='number')

my_transform_dict = {
    type(table_example):dgc.make_df,
    type(input_comp):str
}


def mycb(input_dict):   
    df = input_dict['init_table']
    try:
        if 'x' not in df.columns.values:
            return [[None,None]]
        x = df.x
        scale_y = input_dict['input_box']
        if scale_y is None:
            scale_y = 1
        scale_y = float(str(scale_y))
        y2 = x * scale_y + np.random.normal(size=len(x))*.1
        df2 = pd.DataFrame({'x':x,'y':y2})    
        return [[df2,df2]]
    except Exception as e:
        traceback.print_exc()
        return [[None,None]]
    
# ************ ALL BELOW HERE IS GENERIC ****************
def store_transform_closure(input_comp_ids,input_comp_types,transform_dict,logger):
    def store_transform(input_list):
        if len(input_list)<1:
            dgc.stop_callback(f'store_transform has NO data.  Callback return is ignored',logger)            
#             return {}
        if input_list[0] is None:
            dgc.stop_callback(f'store_transform has NO data.  Callback return is ignored',logger)            
#             return {}
        output_dict = {}
        for i in range(len(input_list)):
            il = input_list[i]
            ict = input_comp_types[i]
            conv_method = str
            conv_key = ict
            if conv_key in transform_dict.keys():
                conv_method = transform_dict[conv_key]
            else:
                logger.warn(f'store_transform unexpected conv_key: {conv_key}')
            conv_data = None if il is None else conv_method(il)
            cid = input_comp_ids[i]
            output_dict[cid] = conv_data
        return output_dict
    return store_transform

input_comps = [table_example,input_comp]
storage_input_tuples = [ic.output_data_tuple for ic in input_comps]
ic_types = [type(ic) for ic in input_comps]
ic_ids = [ic.component_id for ic in input_comps]
stm = store_transform_closure(ic_ids,ic_types,my_transform_dict,logger)
storage_comp = dgc.StoreComponent('store_comp',
        storage_input_tuples,
#         create_data_dictionary_from_df_transformer=stm,
        create_data_dictionary_from_df_transformer=lambda v:v,
#         initial_data={ic.component_id:ic.initial_data for ic in input_comps})
        initial_data={})


def make_comps(input_list,callback,row_comps,row_layout,data_converter):
    input_dict = data_converter(input_list[0])
    if len(input_dict)<1:
        dgc.stop_callback(f'store_transform has NO data.  Callback return is ignored',logger)            
        
    row_data = callback(input_dict)
    if row_data is None:
        dgc.stop_callback(f'store_transform has NO data.  Callback return is ignored',logger)            
        
    all_comps = []
    gc_list = []
    for i in range(len(row_data)):
        data_for_this_row = row_data[i]
        for j in range(len(row_comps)):
            comp = row_comps[j] 
            cid = f'r{i+1}c{j+1}'
            c = comp.clone(cid,initial_data=row_data[i][j],title=cid)
            all_comps.append(c)
        gc_list.append(row_layout)
    html_grid = dgc.create_grid(all_comps,len(row_comps))
    return [html_grid]


# content_div = dgc.ReactiveDiv('variable_content',storage_comp.output_data_tuple,
#                     input_transformer=lambda input_list,_:make_comps(input_list,mycb,my_row_comps,my_row_layout,stm))
content_div = dgc.DivComponent('variable_content',input_component=[storage_comp.output_data_tuple],
                    callback_input_transformer=lambda input_list:make_comps(input_list,mycb,my_row_comps,my_row_layout,stm),
                    logger=logger)

# ac = [title_comp,input_div,input_comp,table_example,xy_example,storage_comp,content_div]
# gc = ['1fr','1fr 1fr','1fr 1fr','1fr', '1fr']
ac = [title_comp,input_div,input_comp,table_example,xy_example,content_div,storage_comp]
gc = ['1fr','1fr 1fr','1fr 1fr','1fr','0%']

app = dgc.make_app(ac,grid_template_columns_list=gc)    
app.run_server(host='127.0.0.1',port=8500)    


2020-01-24 21:07:03,088 - root - INFO - StoreComponent self.output_data_tuple ('store_comp', 'data')


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2020-01-24 21:07:03,113 - werkzeug - INFO -  * Running on http://127.0.0.1:8500/ (Press CTRL+C to quit)
2020-01-24 21:07:04,913 - werkzeug - INFO - 127.0.0.1 - - [24/Jan/2020 21:07:04] "GET / HTTP/1.1" 200 -
2020-01-24 21:07:04,995 - werkzeug - INFO - 127.0.0.1 - - [24/Jan/2020 21:07:04] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-24 21:07:04,999 - werkzeug - INFO - 127.0.0.1 - - [24/Jan/2020 21:07:04] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-24 21:07:05,015 - werkzeug - INFO - 127.0.0.1 - - [24/Jan/2020 21:07:05] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-24 21:07:05,032 - werkzeug - INFO - 127.0.0.1 - - [24/Jan/2020 21:07:05] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.0.0&m=1574289294 HTTP/1.1" 200 -
2020-01-24 21:07:05,033 - werkzeug - INFO - 127.0.0.1 

In [52]:
??dgc.DivComponent

In [53]:
storage_comp.output_data_tuple

('store_comp', 'data')

## Building new reactive class
1. VariableRowDiv defines a div that can have multiple rows that follow a pattern
2. i want to reduce the amount that a python developer has to know about this api.
 * how does markdown really work

In [123]:
DEFAULT_COMPONENT_TRANSFORM_DICT = {
    type(table_example):dgc.make_df,
    type(input_comp):str
}


class VariableRowDiv():
    def __init__(self,
            app_id,
            input_component_list,
            app_callback,
            repeating_row_component_template,
            repeating_row_layout_template,
            component_transform_dict=None,
            logger=None):
        #
        self.app_id = app_id
        self.logger = logger if logger is not None else dgc.init_root_logger('logfile.log','INFO')
        self.app_callback = app_callback
        ctd = DEFAULT_COMPONENT_TRANSFORM_DICT
        if component_transform_dict is not None:
            ctd.update(component_transform_dict)
        self.repeating_row_component_template = repeating_row_component_template
        self.repeating_row_layout_template = repeating_row_layout_template
        #
        input_comps = input_component_list
        storage_input_tuples = [ic.output_data_tuple for ic in input_comps]
        ic_types = [type(ic) for ic in input_comps]
        ic_ids = [ic.component_id for ic in input_comps]
        stm = self.store_transform_closure(ic_ids,ic_types,ctd,self.logger)
        #
        self.storage_comp = dgc.StoreComponent(f'store_comp_{self.app_id}',
                storage_input_tuples,
                create_data_dictionary_from_df_transformer=lambda v:v,
                initial_data={})
        
        mcc = self.make_comps_closure(self.app_callback,
                     self.repeating_row_component_template,
                     self.repeating_row_layout_template,
                     stm,self.logger)
        self.content_div = dgc.DivComponent('variable_content',
                    input_component=[storage_comp.output_data_tuple],
                    callback_input_transformer=lambda input_list:mcc(input_list),
                    logger=self.logger)
        #
        self.final_components = [content_div,storage_comp]
        self.final_layout = ['1fr', '0%']

    def store_transform_closure(self,input_comp_ids,input_comp_types,transform_dict,logger):
        def _store_transform(input_list):
            if len(input_list)<1:
                dgc.stop_callback(f'store_transform has NO data.  Callback return is ignored',logger)            
            if input_list[0] is None:
                dgc.stop_callback(f'store_transform has NO data.  Callback return is ignored',logger)            
            output_dict = {}
            for i in range(len(input_list)):
                il = input_list[i]
                ict = input_comp_types[i]
                conv_method = str
                conv_key = ict
                if conv_key in transform_dict.keys():
                    conv_method = transform_dict[conv_key]
                else:
                    logger.warn(f'store_transform unexpected conv_key: {conv_key}')
                conv_data = None if il is None else conv_method(il)
                cid = input_comp_ids[i]
                output_dict[cid] = conv_data
            return output_dict
        return _store_transform
            
            
    def make_comps_closure(self,callback,row_comps,row_layout,data_converter,logger):
        def make_comps(input_list):
            input_dict = data_converter(input_list[0])
            if len(input_dict)<1:
                dgc.stop_callback(f'store_transform has NO data.  Callback return is ignored',logger)            

            row_data = callback(input_dict)
            if row_data is None:
                dgc.stop_callback(f'store_transform has NO data.  Callback return is ignored',logger)            

            all_comps = []
            gc_list = []
            for i in range(len(row_data)):
                data_for_this_row = row_data[i]
                for j in range(len(row_comps)):
                    comp = row_comps[j] 
                    cid = f'r{i+1}c{j+1}'
                    c = comp.clone(cid,initial_data=row_data[i][j],title=cid)
                    all_comps.append(c)
                gc_list.append(row_layout)
            html_grid = dgc.create_grid(all_comps,len(row_comps))
            return [html_grid]
        return make_comps





In [ ]:
v = VariableRowDiv('my_app',
            input_comps,
            mycb,
            my_row_comps,
            my_row_layout,
            component_transform_dict=None,
            logger=logger)

ac = [title_comp,input_div,input_comp,table_example,xy_example] + v.final_components
gc = ['1fr','1fr 1fr','1fr 1fr'] + v.final_layout

app = dgc.make_app(ac,grid_template_columns_list=gc)    
app.run_server(host='127.0.0.1',port=8500)    


2020-01-24 21:18:13,241 - root - INFO - StoreComponent self.output_data_tuple ('store_comp_my_app', 'data')


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2020-01-24 21:18:13,258 - werkzeug - INFO -  * Running on http://127.0.0.1:8500/ (Press CTRL+C to quit)
2020-01-24 21:18:16,206 - werkzeug - INFO - 127.0.0.1 - - [24/Jan/2020 21:18:16] "GET / HTTP/1.1" 200 -
2020-01-24 21:18:16,385 - werkzeug - INFO - 127.0.0.1 - - [24/Jan/2020 21:18:16] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-24 21:18:16,409 - werkzeug - INFO - 127.0.0.1 - - [24/Jan/2020 21:18:16] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-24 21:18:16,410 - werkzeug - INFO - 127.0.0.1 - - [24/Jan/2020 21:18:16] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-24 21:18:16,415 - werkzeug - INFO - 127.0.0.1 - - [24/Jan/2020 21:18:16] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-24 21:18:16,425 - werkzeug - INFO - 

In [110]:
# This Markdown component allows you to use markdown in html divs
# create row 1

tm_text1 = ''' # Dash DataFrame App '''
tm1 = dgc.dcc.Markdown(tm_text1,style={'color':'white','textAlign': 'center',})
tm_text2 = '''
            1. This app displays Pandas DataFrames, Plotly Graphs and other html components 
            2. like file inputs, dropdown lists, and text input fields, and general divs.

            '''
tm2 = dgc.dcc.Markdown(tm_text2,style={'color':'white','textAlign': 'left',})

st1={
    'line-height': '20px',
    'background-color':'#47bacc',
    'color':'#FFFFF9',
    'vertical-align':'middle',
    'border-style':'hidden'
} 


tm1_div = html.Div([tm1,tm2],style=st1,id='top_div1')
tm2_div = html.Div([tm2],style=st2,id='top_div2')

app = dgc.make_app([tm1_div],grid_template_columns_list=['1fr'])
app.run_server(host='127.0.0.1',port=8500)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2020-01-24 20:51:52,791 - werkzeug - INFO -  * Running on http://127.0.0.1:8500/ (Press CTRL+C to quit)
2020-01-24 20:51:54,663 - werkzeug - INFO - 127.0.0.1 - - [24/Jan/2020 20:51:54] "GET / HTTP/1.1" 200 -
2020-01-24 20:51:54,784 - werkzeug - INFO - 127.0.0.1 - - [24/Jan/2020 20:51:54] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-24 20:51:54,793 - werkzeug - INFO - 127.0.0.1 - - [24/Jan/2020 20:51:54] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-24 20:51:54,798 - werkzeug - INFO - 127.0.0.1 - - [24/Jan/2020 20:51:54] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-24 20:51:54,807 - werkzeug - INFO - 127.0.0.1 - - [24/Jan/2020 20:51:54] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.0.0&m=1574289294 HTTP/1.1" 200 -
2020-01-24 20:51:54,811 - werkzeug - INFO - 127.0.0.1 